In [1]:
from zukilangchain import CustomLM

In [2]:
import os
os.environ['LANGCHAIN_TRACING_V2']='true'
os.environ['LANGCHAIN_ENDPOINT']="https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY']="ls__5c9cc36a095e4130b0c063a09e9dc393"
os.environ['LANGCHAIN_PROJECT']="RAG_test"

In [3]:
from langchain_anthropic import ChatAnthropic
chat = ChatAnthropic(temperature=0, model_name="claude-3-opus-20240229")

In [4]:
llm = CustomLM(api_key="zu-7815d0417fe243b0d216fa9b88df2835", base_url="https://zukijourney.xyzbot.net/v1", model_name="gpt-4-1106-preview")

In [12]:
from langchain.prompts import ChatPromptTemplate

In [5]:
from langchain_community.document_loaders import ArxivLoader


# Arxiv IDs
ids = ["2306.08400", "2302.04761", "2305.18290"]

# Load papers
docs = []
for paper_id in ids:
    doc = ArxivLoader(query=paper_id, load_max_docs=1).load()
    docs.extend(doc)

In [8]:
docs[0].metadata

{'Published': '2023-06-14',
 'Title': 'Simple Embodied Language Learning as a Byproduct of Meta-Reinforcement Learning',
 'Authors': 'Evan Zheran Liu, Sahaana Suri, Tong Mu, Allan Zhou, Chelsea Finn',
 'Summary': "Whereas machine learning models typically learn language by directly training\non language tasks (e.g., next-word prediction), language emerges in human\nchildren as a byproduct of solving non-language tasks (e.g., acquiring food).\nMotivated by this observation, we ask: can embodied reinforcement learning (RL)\nagents also indirectly learn language from non-language tasks? Learning to\nassociate language with its meaning requires a dynamic environment with varied\nlanguage. Therefore, we investigate this question in a multi-task environment\nwith language that varies across the different tasks. Specifically, we design\nan office navigation environment, where the agent's goal is to find a\nparticular office, and office locations differ in different buildings (i.e.,\ntasks). E

In [16]:
system = """
You are an AI assistant that generates high-quality summaries of academic papers or open-source projects for a general audience. Your goal is to distill the key ideas, highlight the most significant contributions, and provide thought-provoking analysis in an engaging, accessible format.

Here are the key elements to include in your summary:

1. A catchy, informative title that captures the main theme
2. A concise overview of the central problem, question or goal of the work
3. Key highlights of the technical approach, innovations or results
4. Reflection on the potential implications and impact of the work

Here is the format you have to use:

-------SUMMARY FORMAT-------
Title: [Title of the Paper]

Technical Details:
- New concepts covered in the paper
    - Subsections as needed
    - Key Insights: [Brief description of the main ideas]
- Approach: [Summary of the technical approach or methodology]
- Sections as needed, look at the example below
- Results: [Key highlights of the results or findings]

Implications:
- Why it matters? [Reflection on the significance and potential impact of the work]
-------END OF FORMAT-------


Use clear, conversational language and avoid overusing jargon or buzzwords. Focus on the most novel and significant aspects of the work.

Here is an example of a high-quality summary to use as inspiration:

-------SUMMARY EXAMPLE-------
Human Feedback Without Reinforcement Learning

"Reinforcement learning from human feedback (RLHF) is widely used to fine-tune pretrained models to deliver outputs that align with human preferences. New work aligns pretrained models without the cumbersome step of reinforcement learning.


RLHF basics: Given a model pretrained to complete sentences in a large text database, reinforcement learning from human feedback proceeds in three steps:

    The model produces pairs of answers to various prompts, and humans rate which of the answers is better.
    Another model learns to mimic how the humans evaluated the outputs. This becomes a so-called reward model.
    The generative model uses evaluations from the reward model to learn, via reinforcement learning, to produce desirable outputs (which earn high rewards) while constrained to keep its answers relatively close to the original model’s output.

Key insight: Instead of training a reward model on human preferences and fine-tuning their language model on the reward model’s output, the authors used the human preferences to fine-tune a copy of their language model directly. The fine-tuning trained the copy to be (i) more likely than the original model to generate human-preferred outputs and (ii) less likely than the original model to generate non-preferred outputs. 

How it works: The authors used DPO to fine-tune a pretrained GPT-J  to summarize text. The dataset was TL;DR.

    The authors prompted GPT-J to produce pairs of outputs. Given a pair of outputs, humans rated which they preferred.
    Given an annotated pair, a copy of GPT-J was trained to generate sequences of tokens for preferred outputs with higher probability than that of the original model, and sequences of tokens for other outputs with low probability than that of the original model. 
    The loss function was constrained to keep the copy from deviating too far from the original model. This step avoided drastic changes that might induce problems such as catastrophic forgetting.

Results: The authors used GPT-4 to estimate whether humans would prefer summaries written by GPT-J fine-tuned via either DPO or RLHF versus human-written summaries. In fine-tuning GPT-J via DPO and RLHF, they experimented with sampling temperatures (a hyperparameter that controls the randomness in choosing the next token, where higher numbers increase randomness) between 0 and 1 and used the best-performing value. GPT-4 evaluated that humans would prefer summaries generated by GPT-J fine-tuned via DPO 61 percent of the time and summaries generated by GPT-J fine-tuned via RLHF 57 percent of the time. In a separate test, human volunteers judged 272 summaries generated by the two models using the best-performing sampling temperatures. The judges preferred the DPO model’s summaries 58 percent of the time.

Why it matters: RLHF is a fundamental technique for making large language models safe for a wide variety of users. Improvements — in this case, a significant boost in efficiency — can help teams to build more useful models, do it faster, and require fewer resources. It’s inspiring that there’s still room for improvement in core LLM building blocks."
-------END OF EXAMPLE-------

Write your summary in around 1000 words. You can use the provided example as a reference to structure your summary effectively.
Your summary must be detailed.
You must not hallucinate, only include facts from the paper, do not make up any information.
"""
human ="""Here is the paper that you have to summarize
------START OF PAPER------
{paper}
------END OF PAPER------
"""

In [17]:
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

summary_generator = prompt | llm
summary_example = summary_generator.invoke({"paper": docs[1].page_content})

In [18]:
summary_example

'-------SUMMARY FORMAT-------\nTitle: Toolformer: The Multi-Tool Language Model\n\nTechnical Details:\n- New concepts covered in the paper\n    - Tool-equipped language models\n    - Self-supervised API call learning\n    - Key Insights: The authors demonstrate that language models can autonomously learn to use external tools such as calculators, search engines, and translation services to improve their performance on various tasks.\n\n- Approach: The authors developed Toolformer, a model that integrates APIs for different functionalities into its predictions, allowing it to perform tasks typically difficult for language models, such as arithmetic or up-to-date factual information retrieval. \n\n- Sampling API Calls: Toolformer samples candidate API calls from within the context of text data.\n- Executing API Calls: The candidate API calls are then executed to return results.\n- Filtering API Calls: A self-supervised loss function assesses the utility of each API call, filtering out un